In [42]:
import numpy as np
import pandas as pd

In [43]:
# Declarando las variables necesarias para luego ponerlas en un DataFrames = [
Tuberias = ['AI', 'BI', 'IJ', 'CJ', 'DJ']

# Longitudes de las tuberías en m
Long_m = [600, 600, 900, 300, 300]

# Diámetros de las tuberías en mm
Diam_mm = [600, 450, 450, 450, 450]

# Facotres de fricción de Darcy propuestos por el ejercicio
f_previo = [0.025, 0.03, 0.03, 0.03, 0.03]

# Coeficiente C en unidades del Sistema Internacional
C_SI = [15.953, 80.67, 121., 40.335, 40.335]

# Rugosidades absolutas de las tuberías en mm
Rugos_mm = [0.03] * len(Tuberias)

# Armando el DataFrame con las variables
df_tubos = pd.DataFrame({
    'Tuberia': Tuberias,
    'Longitud_m': Long_m,
    'Diametro_mm': Diam_mm,
    'Rugos_mm': Rugos_mm,
    'f_previo': f_previo,
    'C_SI': C_SI
})

df_tubos

,Tuberia,Longitud_m,Diametro_mm,Rugos_mm,f_previo,C_SI
0,AI,600,600,0.03,0.025,15.953
1,BI,600,450,0.03,0.030,80.670
2,IJ,900,450,0.03,0.030,121.000
3,CJ,300,450,0.03,0.030,40.335
4,DJ,300,450,0.03,0.030,40.335


In [44]:
# Armando un dataframe con las alturas de los tanques de almacenamiento
Tanques = ['A', 'B', 'C', 'D']
Alturas_m = [80., 90., 100., 90.]

df_tanques = pd.DataFrame({
    'Tanque': Tanques,
    'Altura_m': Alturas_m
})

df_tanques

,Tanque,Altura_m
0,A,80.0
1,B,90.0
2,C,100.0
3,D,90.0


In [45]:
# Montando la matriz de ecuaciones de conservación de masa y energía 
Mat_coef = np.zeros((len(Tuberias) + 2, len(Tuberias) + 2))

# Conservación de masa en los nodos

# Nodo I: Sale hacia A, entra desde B y entra desde J
Mat_coef[0, 0] = 1
Mat_coef[0, 1] = -1
Mat_coef[0, 2] = -1

# Nodo J (los caudales de los tanques entran y el que va al nodo, sale)
Mat_coef[1, 2] = 1
Mat_coef[1, 3] = -1
Mat_coef[1, 4] = 1

# Conservaciones de energía que quedan fijas en la matriz y se pueden declarar
Mat_coef[2, 5] = 1
Mat_coef[3, 5] = 1
Mat_coef[4, 5] = 1
Mat_coef[4, 6] = -1
Mat_coef[5, 6] = 1
Mat_coef[6 ,6] = 1

# Definiendo una función para calcular las otras posiciones de la matriz usando
# el coeficiente C
def coeficiente_C(A, df_tubos, Q):
    """
    Calcula los coeficientes en la matriz A corresponidentes a la pérdida de 
    carga
    A: Matriz de coeficientes de la tubería.
    df_tubos: DataFrame que contiene las propiedades de las tuberías.
    """

    # Nodo I con tubería AI (fila 2)
    A[2, 0] = -df_tubos['C_SI'][0] * np.abs(Q[0])

    # Nodo I con tubería BI (fila 3)
    A[3, 1] = df_tubos['C_SI'][1] * np.abs(Q[1])

    # Nodo I con tubería IJ (fuila 4)
    A[4, 2] = df_tubos['C_SI'][2] * np.abs(Q[2])

    # Nodo J con tubería CJ (fila 5)
    A[5, 3] = df_tubos['C_SI'][3] * np.abs(Q[3])

    # Nodo J con tubería DJ (fila 6)
    A[6, 4] = -df_tubos['C_SI'][4] * np.abs(Q[4])
    
    return A

# Definiendo caudales de las tuberías como 0.1 m3/s
Q = [0.1, 0.1, 0.1, 0.1, 0.1]

# Llenando la matriz de coeficientes con los valores de C
Mat_coef = coeficiente_C(Mat_coef, df_tubos, Q)

# Imprimiendo la matriz para revisar su contenido
np.round(Mat_coef, 3)

array([[ 1.   , -1.   , -1.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  1.   , -1.   ,  1.   ,  0.   ,  0.   ],
       [-1.595,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ],
       [ 0.   ,  8.067,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ],
       [ 0.   ,  0.   , 12.1  ,  0.   ,  0.   ,  1.   , -1.   ],
       [ 0.   ,  0.   ,  0.   ,  4.034,  0.   ,  0.   ,  1.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   , -4.034,  0.   ,  1.   ]])

In [46]:
# Definiendo el vector de mano derecha
b = np.zeros(len(Tuberias) + 2)

b[2] = df_tanques['Altura_m'][0]
b[3] = df_tanques['Altura_m'][1]
b[5] = df_tanques['Altura_m'][2]
b[6] = df_tanques['Altura_m'][3]

# Imprimiendo el vector de mano derecha
b

array([  0.,   0.,  80.,  90.,   0., 100.,  90.])

In [47]:
# Defniendo los parámetros numéricos para la resolución del sistema
tol = 1e-3
err = 1e10
count = 1

# Armando un vector inicial de caudales y alturas supuestas
Q0 = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 85., 95.])

while err > tol: 

    # Calculando la matriz de coeficientes con los caudales actuales
    Mat_coef = coeficiente_C(Mat_coef, df_tubos, Q0)

    # Resolviendo el sistema de ecuaciones
    Q1 = np.linalg.solve(Mat_coef, b)

    # Calculando el error como la norma del vector de caudales
    err = np.linalg.norm(Q1 - Q0)

    # Actualizando los caudales para la siguiente iteración
    Q0 = 0.5 * (Q0 + Q1)

    # Imprimiendo el estado actual de los caudales y el error
    print(f"\nIteración {count}:")
    print("Matriz de coeficientes:\n", np.round(Mat_coef, 4))
    print(f"Caudales y alturas = {np.round(Q1, 4)}\nError = {err:.4f}")
    
    count += 1


Iteración 1:
Matriz de coeficientes:
 [[ 1.     -1.     -1.      0.      0.      0.      0.    ]
 [ 0.      0.      1.     -1.      1.      0.      0.    ]
 [-1.5953  0.      0.      0.      0.      1.      0.    ]
 [ 0.      8.067   0.      0.      0.      1.      0.    ]
 [ 0.      0.     12.1     0.      0.      1.     -1.    ]
 [ 0.      0.      0.      4.0335  0.      0.      1.    ]
 [ 0.      0.      0.      0.     -4.0335  0.      1.    ]]
Caudales y alturas = [ 1.7564  0.8923  0.8641  1.6717  0.8076 82.8019 93.2574]
Error = 3.8463

Iteración 2:
Matriz de coeficientes:
 [[  1.      -1.      -1.       0.       0.       0.       0.    ]
 [  0.       0.       1.      -1.       1.       0.       0.    ]
 [-14.8073   0.       0.       0.       0.       1.       0.    ]
 [  0.      40.0238   0.       0.       0.       1.       0.    ]
 [  0.       0.      58.3271   0.       0.       1.      -1.    ]
 [  0.       0.       0.      35.73     0.       0.       1.    ]
 [  0.       0.   

In [ ]:
# Ahora vamos a implementar el cálculo de las pérdidas, pero no con C, sino con 
# el factor de fricción de Darcy. Casi todo el proceso es igual, pero cambia el 
# cálculo de los coeficientes de la matriz donde se ponen las pérdidas de carga.